In [1]:
import numpy as  np

from keras.layers import Dense, Conv1D, Conv2D, Flatten, Input, Concatenate, Dropout, Subtract, LSTM
from keras.models import Model

Using TensorFlow backend.


In [2]:
all_training_queries = np.load('data/all_training_queries.npy')
all_training_songs = np.load('data/all_training_songs.npy')
all_training_labels = np.load('data/all_training_labels.npy')

all_testing_queries = np.load('data/all_testing_queries.npy')
all_testing_songs = np.load('data/all_testing_songs.npy')
all_testing_labels = np.load('data/all_testing_labels.npy')

In [ ]:
embedder_conv1d_1 = Conv1D(filters = 15, kernel_size = 15, strides = 3, input_shape = (20700,19))
embedder_conv1d_2 = Conv1D(filters = 12, kernel_size = 8, strides = 3)
embedder_conv1d_3 = Conv1D(filters = 9, kernel_size = 8, strides = 3)
embedder_conv1d_4 = Conv1D(filters = 5, kernel_size = 8, strides = 3)
embedder_conv1d_5 = Conv1D(filters = 2, kernel_size = 8, strides = 3)

q = Input(shape = (20700,19,))
s = Input(shape = (20700,19,))

q_embeddings = embedder_conv1d_1(q)
q_embeddings = embedder_conv1d_2(q_embeddings)
q_embeddings = embedder_conv1d_3(q_embeddings)
q_embeddings = embedder_conv1d_4(q_embeddings)
q_embeddings = embedder_conv1d_5(q_embeddings)
q_embeddings = Flatten()(q_embeddings)

s_embeddings = embedder_conv1d_1(s)
s_embeddings = embedder_conv1d_2(s_embeddings)
s_embeddings = embedder_conv1d_3(s_embeddings)
s_embeddings = embedder_conv1d_4(s_embeddings)
s_embeddings = embedder_conv1d_5(s_embeddings)
s_embeddings = Flatten()(s_embeddings)

final = Concatenate(axis = -1)([q_embeddings, s_embeddings])
#final = Subtract()([q_embeddings, s_embeddings])
label = Dense(50, activation = 'sigmoid')(final)
label = Dropout(rate = 0.25)(label)
label = Dense(1, activation = 'sigmoid')(label)

#print(q_embeddings.shape)

In [4]:
embedder_lstm_1 = LSTM(15, input_shape = (20700,19,), return_sequences = True)
embedder_lstm_2 = LSTM(13)

q = Input(shape = (20700,19,))
s = Input(shape = (20700,19,))

q_embeddings = embedder_lstm_1(q)
q_embeddings = embedder_lstm_2(q_embeddings)

s_embeddings = embedder_lstm_1(s)
s_embeddings = embedder_lstm_2(s_embeddings)

final = Concatenate(axis = -1)([q_embeddings, s_embeddings])
#final = Subtract()([q_embeddings, s_embeddings])
label = Dense(50, activation = 'sigmoid')(final)
label = Dropout(rate = 0.25)(label)
label = Dense(1, activation = 'sigmoid')(label)

In [5]:
model = Model(inputs = [q, s], outputs = label)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [6]:
model.fit([all_training_queries, all_training_songs], all_training_labels, validation_split = 0.2, epochs = 100, batch_size = 64)

Train on 283 samples, validate on 71 samples
Epoch 1/100
192/283 [===================>..........] - ETA: 3:43 - loss: 0.9882 - acc: 0.3073

KeyboardInterrupt: 

In [ ]:
model.save('HummingNet.h5')

In [ ]:
all_testing_queries = np.load('data/all_testing_queries.npy')
all_testing_songs = np.load('data/all_testing_songs.npy')
all_testing_labels = np.load('data/all_testing_labels.npy')

In [ ]:
model.predict([all_testing_queries, all_testing_songs])